# Component Separation on CMB S4 and QUBIC+

In [1]:
import qubicplus
import CMBS4
import pysm3
import pysm3.units as u
from pysm3 import utils
import numpy as np
from qubic import camb_interface as qc
import healpy as hp
import matplotlib.pyplot as plt
import os
import random as rd
import string
import qubic
from importlib import reload

center = qubic.equ2gal(0, -57)
# If there is not this command, the kernel shut down every time..
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

nside=256
covmap = hp.ud_grade(hp.read_map('QUBIC-Typical-Coverage.fits'), nside)
skyconfig = {'cmb':42, 'dust':'d0'}

thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/healpy/fitsfunc.py:368: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  warnings.warn(
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/healpy/fitsfunc.py:391: UserWarning: NSIDE = 128
  warnings.warn("NSIDE = {0:d}".format(nside))
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/healpy/fitsfunc.py:400: UserWarning: ORDERING = RING in fits file
  warnings.warn("ORDERING = {0:s} in fits file".format(ordering))
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/healpy/fitsfunc.py:428: UserWarning: INDXSCHM = IMPLICIT
  warnings.warn("INDXSCHM = {0:s}".format(schm))


In [134]:
reload(qubicplus)
reload(CMBS4)

nf=5
index=8

maps_s4, _, noise_s4, nus_s4, fwhm_s4=CMBS4.S4(skyconfig, nf).get_fullsky_S4(verbose=False, iib=False, coverage=None)
maps_qp, _, noise_qp, nus_qp, fwhm_qp, depthi, depthp=qubicplus.QUBICplus(skyconfig, nf).get_fullsky_qubicplus(index=index, verbose=False, 
                                                                                  coverage=covmap, iib=False)

/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/healpy/fitsfunc.py:197: FutureWarning: The default dtype of write_map() will change in a future version: explicitly set the dtype if it is important to you
  warnings.warn(
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/pysm3/utils/data.py:37: UserWarning: Access data from /tmp/XnX8l38i8O
  warnings.warn(f"Access data from {full_path}")
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/pysm3/utils/data.py:42: UserWarning: Retrieve data for pysm_2/dust_t_new.fits (if not cached already)
  warnings.warn(f"Retrieve data for {filename} (if not cached already)")
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/pysm3/utils/data.py:42: UserWarning: Retrieve data for pysm_2/dust_q_new.fits (if not cached already)
  warnings.warn(f"Retrieve data for {filename} (if not cached already)")
/Users/mathiasregnier/miniconda3/lib/python3.8/site-packages/pysm3/utils/data.py:42: UserWarning: Retrieve data fo

In [135]:
print(nus_qp, fwhm_qp)

[246.24 258.12 270.   281.88 293.76] [0.2375731  0.22663877 0.21666667 0.20753512 0.19914216]


In [136]:
import fgbuster as fgb
from fgbuster import basic_comp_sep, get_instrument

def reconvolve(maps, fwhms, ref_fwhm, verbose=False):
    if verbose:
        print('Reconvolution to common FWHM')
    sig_conv = np.sqrt(ref_fwhm**2 - fwhms**2)
    maps_out = np.zeros_like(maps)
    for i in range(len(fwhms)):
        if sig_conv[i] == 0:
            if verbose:
                print('Map {0:} fwhmin={1:6.3f} fwhmout={2:6.3f} => We do not reconvolve'.format(i,
                                                                                             fwhms[i], ref_fwhm))
            maps_out[i,:] = maps[i,:]
        else:
            if verbose:
                print('Map {0:} fwhmin={1:6.3f} fwhmout={2:6.3f} => We reconvolve with {3:6.3f}'.format(i,
                                                                                                    fwhms[i],
                                                                                                    ref_fwhm,
                                                                                                    sig_conv[i]))
            maps_out[i,:] = hp.smoothing(maps[i,:], fwhm=np.deg2rad(sig_conv[i]), pol=True, verbose=False)
    return maps_out

def give_me_rms_I(X, nside) :
    rms = np.std(X[:, 0, :], axis=1)
    rms *= np.sqrt(hp.nside2pixarea(nside, degrees=True)) * 60
    return rms

In [137]:
maps_s4_reconv = reconvolve(maps_s4, fwhm_s4, np.max(fwhm_s4)+0.01)
maps_qp_reconv = reconvolve(maps_qp, fwhm_qp, np.max(fwhm_qp)+0.01)

In [138]:
from fgbuster import (separation_recipes, xForecast, CMB, Dust, Synchrotron, FreeFree, PowerLaw,  # sky-fitting model
                      basic_comp_sep)

comp = [Dust(nu0=150., temp=20), CMB()]

instr = get_instrument('Qubic')
instr.frequency = nus_s4
instr.fwhm = fwhm_s4
instr.depth_i = depthi
instr.depth_p = depthp

<ipython-input-138-4db802bd3394>:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.frequency = nus_s4
<ipython-input-138-4db802bd3394>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.fwhm = fwhm_s4
<ipython-input-138-4db802bd3394>:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.depth_i = depthi
<ipython-input-138-4db802bd3394>:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.depth_p = depthp


In [139]:
#maps_s4_reconv[:, :, ~pixok] = hp.UNSEEN

r = basic_comp_sep(comp, instr, maps_s4_reconv)

In [140]:
r.x

array([1.53998224])

In [141]:
from fgbuster import (separation_recipes, xForecast, CMB, Dust, Synchrotron, FreeFree, PowerLaw,  # sky-fitting model
                      basic_comp_sep)

comp = [Dust(nu0=150., temp=20), CMB()]

instr = get_instrument('Qubic')
instr.frequency = nus_qp
instr.fwhm = np.ones(nf)*(np.max(fwhm_qp)+0.01)
instr.depth_i = np.ones(nf)*depthi[index]*np.sqrt(nf)
instr.depth_p = np.ones(nf)*depthp[index]*np.sqrt(nf)

<ipython-input-141-b067a9a60d2b>:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.frequency = nus_qp
<ipython-input-141-b067a9a60d2b>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.fwhm = np.ones(nf)*(np.max(fwhm_qp)+0.01)
<ipython-input-141-b067a9a60d2b>:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.depth_i = np.ones(nf)*depthi[index]*np.sqrt(nf)
<ipython-input-141-b067a9a60d2b>:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  instr.depth_p = np.ones(nf)*depthp[index]*np.sqrt(nf)


In [142]:
maps_qp_reconv[:, :, ~pixok] = hp.UNSEEN

r = basic_comp_sep(comp, instr, maps_qp_reconv, method='L-BFGS-B', options={'eps': 1e-10, 'ftol': 1e-10})

In [143]:
r.x

array([-0.45792282])